# AH Bonnen parser

In [2]:
import os
import re
import io
import PyPDF2
from PIL import Image
import pandas as pd
from pdf2image import convert_from_path
import pytesseract

import warnings
warnings.filterwarnings("ignore")

In [6]:
def receipt_renamer(folder_path):
    """
    Function renames the download ah receipts to a more sortable name
    Input: A path to a folder with ah receipt pdfs. 
    """

    # Get a list of all the files in the directory
    files = os.listdir(folder_path)
    # Loop through each file in the directory
    for file in files:
        # Use regular expressions to extract the date from the filename
        match = re.search(r'\d{2}-\d{2}-\d{2}', file)
        if match:
            # Extract the date from the match object
            date = match.group()
            # Convert the date to the desired format (YYYY-MM-DD)
            new_name = re.sub(r'(\d{2})-(\d{2})-(\d{2})', r'20\3-\2-\1', date) + '_AH_kassabon'
            # Rename the file with the new name
            try:
                os.rename(os.path.join(folder_path, file), os.path.join(folder_path, new_name + '.pdf'))
            except Exception as E:
                print(E)
    print('succesfully renamed all files')

In [10]:
def pdf_to_img(pdf_path, pdf_img_path):

    # Get a list of all the files in the directory
    files = os.listdir(pdf_path)
    # Loop through each file in the directory
    for pdf_file in files:

        full_path = os.path.join(pdf_path, pdf_file)
        # Convert the first page of the PDF file to a PIL image
        images = convert_from_path(full_path, first_page=1, last_page=1)
        image = images[0]
        # obtain the pdf name to name the image.
        pdf_file = pdf_file.replace('.pdf','.jpg')
        # Save the image to a file
        image.save(pdf_img_path + pdf_file)
        
    print('succesfully converted all pdf files to images')

In [11]:
#pdf_to_img('ah_bonnen_pdf', 'ah_bonnen_img\\')

succesfully converted all pdf files to images


In [12]:
def text_to_df(text, img_file):

    temp_df = pd.DataFrame(columns=['Amount','Product','Price'])
    try:
        text_without_start = text.split('BONUSKAART xx9644', 1)[1]
        text_without_end = text_without_start.split('UW VOORDEEL',1)[0]
        lines = text_without_end.split("\n")
    except Exception as ex:
        print(img_file, ' did not work because', ex)
        return temp_df

    match = re.search(r"\d{1,2}-\d{1,2}-\d{4}", text)
    if match:
        date = match.group()
        date_without_time = re.sub(r"\s+\d{1,2}:\d{1,2}\s*", " ", date)

    for line in lines:
        # Create an I that decides which part should be the amount.
        i = -1
        if line:
            parts = line.split()
            amount = parts[0]

            # Ignore the bonus box lines.
            if 'BBOX' in amount or 'BONUS' in amount:
                continue
            # The bonus and bonux box lines are the last on receipt.
            # Therefore they have to be ignored to get the price.
            while parts[i] == 'B' or parts[i] == 'BB':
                i -=1
            
            # If the amount of the product is expressed in KG, we want to grab the kg price
            # And remove KG from the line
            if 'KG' in amount:
                i -=1
                product_name = " ".join(parts[1:i])
                amount = amount.replace('KG','')
            else:
                product_name = " ".join(parts[1:i])

            # Assign price and make it a float
            # Also split price if the value is not right
            price = parts[i]
            price = price.replace(',', '.')
            try:
                if float(price) > 100:
                    price = price[:1] + '.' + price[1:]
                price = float(price)
            except Exception as ex2:
                print('Encounterd: ', ex2, 'For ', product_name, 'In', date_without_time)
                price = 999

            temp_df.loc[len(temp_df)] = [amount, product_name, price]

    temp_df['Amount'] = pd.to_numeric(temp_df['Amount'], errors='coerce').fillna(1)
    temp_df['Date'] = pd.to_datetime(date_without_time, format='%d-%m-%Y').date()
    # Multiple the amount bought by the price to get the right price
    # Known issue: Messes up subtotaal.
    # If price is not a float it will return None
    temp_df['Price_Amount'] = temp_df.apply(lambda x: round(x['Price'] * x['Amount'], 2) if isinstance(x['Price'], float) else None, axis=1)

    return temp_df 

In [13]:
def text_from_img(pdf_img_path):

    df = pd.DataFrame(columns=['Amount','Product','Price','Date'])

    # Get a list of all the files in the directory
    files = os.listdir(pdf_img_path)
    # Loop through each file in the directory
    for img_file in files:

        full_path = os.path.join(pdf_img_path, img_file)
        image = Image.open(full_path)
        # Convert the image to grayscale
        image = image.convert('L')
        # Process the image with pytesseract
        text = pytesseract.image_to_string(image)
        temp_df = text_to_df(text, img_file)
        df = pd.concat([df, temp_df]).reset_index(drop=True)

    return df


In [15]:
img_path = 'C:\\Users\\rie12\\Documents\\GitHub\\AH_kassabonnen\\ah_bonnen_img'

In [16]:
test = text_from_img(img_path)

2001-03-22_AH_kassabon.jpg  did not work because list index out of range
Encounterd:  could not convert string to float: '=' For  GOUDSALAMI
Encounterd:  could not convert string to float: 'GOUDSALAMI' For  
Encounterd:  could not convert string to float: 'LAS' For  Gl PENNE
Encounterd:  could not convert string to float: '—' For  AH ROAST HAM
Encounterd:  could not convert string to float: '-' For  GOUDSALAMI
Encounterd:  could not convert string to float: 'i' For  KIPFILET
2022-01-07_AH_kassabon.jpg  did not work because list index out of range
Encounterd:  could not convert string to float: 'ia' For  AH YORKHAM
Encounterd:  could not convert string to float: '35%' For  KL SCHIJFJES 0,65
Encounterd:  could not convert string to float: 'Ly' For  DONUT SUIKER
2022-05-04_AH_kassabon.jpg  did not work because list index out of range
Encounterd:  could not convert string to float: 'L§5' For  UNOX GOODN
2022-06-22_AH_kassabon.jpg  did not work because list index out of range
Encounterd:  c

In [22]:
test = test.sort_values(by='Date')

In [23]:
test

,Amount,Product,Price,Date,Price_Amount
104,5.0,SUBTOTAAL,9.54,2021-10-11,47.70
103,1.0,TIJGER VOLK,1.02,2021-10-11,1.02
102,1.0,PASTASAUS,1.79,2021-10-11,1.79
101,1.0,AH SPAGHETTI,2.25,2021-10-11,2.25
100,1.0,ZAANSE HOEVE,1.49,2021-10-11,1.49
...,...,...,...,...,...
1967,1.0,TAKS! TROPIS,1.69,2023-04-30,1.69
1966,1.0,DESPERADOS,9.79,2023-04-30,9.79
1965,1.0,OETKER PIZZA,4.19,2023-04-30,4.19
1972,1.0,LAY'S OVEN B,2.29,2023-04-30,2.29


# AH Bonnen grafieken

Wat wil ik laten zien:

* Totale Prijs verschil over tijd
* Meest gekochte producten
* Voor meest gekochte producten het verschil over tijd

In [24]:
import matplotlib.pyplot as plt 
import plotly.express as px
import plotly.graph_objects as go

In [25]:
# Totale prijs df maken.
test['Price'] = pd.to_numeric(test['Price'])
total_df = test[test['Product'] == 'SUBTOTAAL']

In [29]:
# Data klaar maken.
total_df.loc[:, 'divided_price'] = total_df['Price'] / total_df['Amount']
total_df['Date'] = pd.to_datetime(total_df['Date'])

# Calculate rolling mean for Price column
total_df['Rolling Mean'] = total_df['Price'].rolling(window=10).mean()

In [30]:
line1 = go.Scatter(x=total_df['Date'], y=total_df['Price'], name='Price over Time')
line2 = go.Scatter(x=total_df['Date'], y=total_df['Rolling Mean'], name='10 visits rolling mean')
#line2 = go.Scatter(x=total_df['Date'], y=total_df['Amount'], name='Amount over Time')
data = [line1, line2]
fig = go.Figure(data=data)
fig.show()

## Meest voorkomende producten

In [134]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Amount        322 non-null    float64
 1   Product       322 non-null    object 
 2   Price         322 non-null    float64
 3   Date          322 non-null    object 
 4   Price_Amount  321 non-null    float64
dtypes: float64(3), object(2)
memory usage: 12.7+ KB


In [135]:
test

,Amount,Product,Price,Date,Price_Amount
0,1.0,AH ICETEA,0.89,2023-01-03,0.89
1,1.0,OETKER PIZZA,4.19,2023-01-03,4.19
2,1.0,JELLY BEANS,1.29,2023-01-03,1.29
3,1.0,LAY'S OVEN B,2.09,2023-01-03,2.09
4,1.0,PEPPERMINT,1.39,2023-01-03,1.39
...,...,...,...,...,...
317,1.0,PAPRIKA,1.39,2023-04-30,1.39
318,1.0,AH KAISERBR,0.29,2023-04-30,0.29
319,1.0,AH ITAL BOL,0.79,2023-04-30,0.79
320,1.0,ZAANS BRUIN,1.09,2023-04-30,1.09


In [136]:
most_common = test.groupby('Product')['Product'].count()

In [137]:
df_grouped = test.groupby('Product').agg({'Amount': 'sum', 'Product': 'size'})
df_grouped = df_grouped.rename(columns={'Product':'Count'})
df_grouped = df_grouped.reset_index(drop=False)
df_grouped = df_grouped.sort_values(by='Count', ascending=False)

In [138]:
df_grouped

,Product,Amount,Count
146,SUBTOTAAL,300.0,26
166,ZAANS BRUIN,12.0,12
113,OETKER PIZZA,12.0,12
102,LAY'S OVEN B,10.0,10
123,PICKWICK,10.0,10
...,...,...,...
71,DUYWIS NOOT,1.0,1
70,DUIS,1.0,1
69,DORITOS170-1,1.0,1
67,DM PERZIK,1.0,1


# Individual product graphs

In [149]:
def graph_maker(df, product_name):
        df = df[df['Product'] == product_name]
        fig = px.line(df, x='Date', y='Price', title=f'Price for {product_name} over Time')
        return fig

In [151]:
graph_maker(test, 'OETKER PIZZA')